# 네이버 영화 리뷰 감성 분류

## 1. 네이버 영화 리뷰 데이터 전처리

In [1]:
import numpy as np
import pandas as pd

In [2]:
import re
from konlpy.tag import Okt

In [3]:
train_data= pd.read_table('data/ratings_train.txt')
test_data= pd.read_table('data/ratings_test.txt')

In [4]:
# Null 값을 가진 샘플을 제거
train_data=train_data.dropna(how='any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [5]:
test_data=test_data.dropna(how='any') # Null 값이 존재하는 행 제거
print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [6]:
# 한글과 공백을 제외하고 모두 제거하는 정규 표현식
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [7]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


### 훈련 데이터와 테스트 데이터에 대해 다음을 수행
- 토큰화와 불용어 제거
- 사이킷 런의 CountVectorizer가 처리할 수 있는 형태로 변환

In [8]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

In [10]:
import tqdm.notebook as tn
# 훈련 데이터
X_train=[]
for sentence in tn.tqdm(train_data['document']):
    morphs = okt.morphs(sentence, stem=True) # 토큰화
    temp = ' '.join(morph for morph in morphs if not morph in stopwords)
    X_train.append(temp)

In [11]:
# 테스트 데이터
X_test=[]
for sentence in tn.tqdm(test_data['document']):
    morphs = okt.morphs(sentence, stem=True) # 토큰화
    temp = ' '.join(morph for morph in morphs if not morph in stopwords)
    X_test.append(temp)

In [12]:
y_train=np.array(train_data['label'])
y_test=np.array(test_data['label'])

## 3. Naive Bayes 분류기로 분류하기

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB  # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score     # 정확도 계산

In [14]:
# DTM - Document Term Matrix
dtmvector = CountVectorizer()
dtmvector.fit(X_train)
X_train_dtm = dtmvector.transform(X_train)
print(X_train_dtm.shape)

(149995, 42118)


In [15]:
# 사이킷 런에서 제공하는 나이브 베이즈 모델
model = MultinomialNB()
model.fit(X_train_dtm, y_train)

MultinomialNB()

In [16]:
# 테스트 데이터 DTM 형태로 변환
X_test_dtm = dtmvector.transform(X_test)
print(X_test_dtm.shape)

(49997, 42118)


In [17]:
predicted = model.predict(X_test_dtm) #테스트 데이터에 대한 예측
print("정확도: %.4f" % accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8248


In [18]:
import joblib
joblib.dump(model, 'model/movie_nb.pkl')

['model/movie_nb.pkl']

In [19]:
joblib.dump(dtmvector, 'model/movie_nb_dtm.pkl')

['model/movie_nb_dtm.pkl']